In [1]:
import xarray as xr
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import pendulum
import geopandas as gpd
from dateutil.relativedelta import relativedelta
import warnings
warnings.filterwarnings("ignore")

from skill import config
from skill import recorte
from skill import plot
from skill import outros

In [2]:
dir_dados = config.dir_dados
dir_img = config.dir_img
dir_shp = config.dir_shp

In [3]:
br_madeira = gpd.read_file(f'{dir_shp}/brasil_madeira.shp')

In [4]:
meses = ['', 'Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho', 'Julho', 'Agosto', 'Setembro', 'Outubro', 'Novembro','Dezembro']

In [6]:
for centro in ['cfsv2', 'cmc1', 'cmc2', 'nasa', 'gfdl','gfdl_flor', 'ncar_ccsm4', 'nmme']:

    dir_img.joinpath(centro).mkdir(exist_ok=True)

    for mes in np.arange(1,13,1):

        date = pendulum.now().set(year=1975, month=mes, day=21)

        ds = outros.download(centro, mes)
        ds_recortado = recorte.recortar(ds, br_madeira)

        for lead in np.arange(1,4,1):

            var = f'sk{lead}'
            lead_mes = (date + relativedelta(months=lead)).month

            fig,ax = plt.subplots(1,1, figsize=(10,10), subplot_kw=dict(projection = ccrs.PlateCarree()))

            plt.suptitle(f"Skill {centro}", x=0.45, y=0.875, fontsize=16)
            ax.set_title(f'CI e CC: {meses[mes]}', loc='left')
            ax.set_title(f'Válido para {meses[lead_mes]}', loc='right')
            ax.text(-41.5,-33,'Lammoc - UFF', fontstyle='italic')

            plot.plot_mesh(ax, ds_recortado[var].isel(time=0), vmin=-1, vmax=1)

            plt.close('all')

            fig.savefig(dir_img.joinpath(centro, f'skill_{centro}-init{mes}_lead{lead}.png'), bbox_inches='tight', facecolor='w', edgecolor='w')